In [ ]:
# Install the torchinfo package for showing the network architecture information
!pip install torchinfo -qqq
!pip install opencv-python
!pip install ultralytics
!pip install roboflow

import torch
import torchvision
from torchvision import datasets,transforms
import torch.nn as nn
import torch.nn.functional as F
from torch import nn,optim,no_grad
from torch.utils.data import DataLoader
from torchinfo import summary

import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import pickle
import sys
import time

from pycocotools.coco import COCO
from PIL import Image, ImageDraw

import json
import os
from pathlib import Path
import shutil
from tqdm import tqdm
import cv2
import yaml
import shutil
from ultralytics import YOLO

import datetime
import logging
from sklearn.metrics import precision_score, recall_score, f1_score

import PIL.Image
import io
from IPython.display import clear_output, Image, display, HTML
import ipywidgets as widgets
import PIL.Image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.4/982.4 kB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set device to GPU if available
has_mps = torch.backends.mps.is_built()
device = "cuda" if torch.cuda.is_available() else "mps" if has_mps else "cpu"
print(f"Python version: {sys.version_info.major, sys.version_info.minor, sys.version_info.micro}")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

Mounted at /content/drive
Python version: (3, 11, 12)
PyTorch version: 2.6.0+cu124
Device: cuda


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="TSNpeVNs514e85awM4WA")
project = rf.workspace("pg-g-sgpnk").project("csgo-wlnz5")
version = project.version(1)
dataset = version.download("yolov11")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to CSGO-1 in yolov11:: 100%|██████████| 1612/1612 [00:00<00:00, 2541.58it/s]


In [ ]:
import os
import cv2
import torch
from ultralytics import YOLO
import shutil

WORK_DIR = "dataset"
DATA_DIR = "/content/CSGO-1"
CONFIG_FILE = "data.yaml"

if not os.path.exists(WORK_DIR):
    os.makedirs(WORK_DIR)

def create_config():
    config_content = f"""
train: {DATA_DIR}/train/images
val: {DATA_DIR}/valid/images
test: {DATA_DIR}/test/images
nc: 2
names: ['CT', 'T']
    """
    with open(f"{WORK_DIR}/{CONFIG_FILE}", "w") as f:
        f.write(config_content.strip())
    print(f"Configuration file created: {WORK_DIR}/{CONFIG_FILE}")

# Helen's new functions
def backup_model_and_config(epoch, model_weights_path, config_file_path, backup_dir="backup"):
    os.makedirs(backup_dir, exist_ok=True)
    backup_epoch_dir = os.path.join(backup_dir, f"epoch_{epoch}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}")
    os.makedirs(backup_epoch_dir, exist_ok=True)
    shutil.copy(model_weights_path, os.path.join(backup_epoch_dir, "best.pt"))
    shutil.copy(config_file_path, os.path.join(backup_epoch_dir, "config.yaml"))
    print(f"Backup of model and config completed: {backup_epoch_dir}")


def setup_logging(log_file="training.log"):
    logging.basicConfig(filename=log_file, level=logging.INFO,
                        format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info("Logging initialized.")


def log_training_metrics(epoch, loss, precision, recall, f1):
    logging.info(f"Epoch: {epoch}, Loss: {loss:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

def evaluate_model(predictions, targets):
    print(f"Total samples: {len(predictions)}")
    print(f"Number of -1 predictions (no detection): {predictions.count(-1)}")

    # Create masks for valid predictions and targets
    valid_indices = [i for i, (pred, target) in enumerate(zip(predictions, targets))
                    if pred != -1 and target != -1]

    if not valid_indices:
        logging.warning("No valid prediction-target pairs for evaluation")
        return

    filtered_preds = [predictions[i] for i in valid_indices]
    filtered_targets = [targets[i] for i in valid_indices]

    print(f"Valid samples for evaluation: {len(filtered_preds)}")

    # Now we can safely calculate metrics
    try:
        # Calculate and print the classification report
        report = classification_report(filtered_targets, filtered_preds, target_names=['CT', 'T'])
        print(report)

        # Calculate precision, recall, and F1 score
        y_true = np.array(filtered_targets)
        y_pred = np.array(filtered_preds)
        precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

        # Log the training metrics
        avg_loss = 0.0  # Replace with actual loss if available
        log_training_metrics(0, avg_loss, precision, recall, f1)
        print(f"Evaluation results - Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
    except Exception as e:
        logging.error(f"Error during evaluation: {str(e)}")
        print(f"Error during evaluation: {str(e)}")
        print(f"Prediction classes: {set(filtered_preds)}")
        print(f"Target classes: {set(filtered_targets)}")

def run_evaluation(model):
    # Load validation targets
    val_targets = []
    for label_file in os.listdir(f"{DATA_DIR}/valid/labels"):
        with open(os.path.join(f"{DATA_DIR}/valid/labels", label_file), 'r') as f:
            for line in f:
                class_id = int(line.split()[0])
                val_targets.append(class_id)

    # Perform predictions on validation images
    val_images = [os.path.join(DATA_DIR, "valid/images", img) for img in os.listdir(f"{DATA_DIR}/valid/images")]
    predictions = []
    for img_path in val_images:
        result = model.predict(source=img_path, conf=0.5)
        predicted_class = result[0].boxes.cls[0].item() if result[0].boxes.shape[0] > 0 else -1
        predictions.append(predicted_class)

    # Call the evaluation function
    evaluate_model(predictions, val_targets)



def train_model():
    model = YOLO("yolo11n.pt")
    model.train(
        data=f"{WORK_DIR}/{CONFIG_FILE}",
        epochs=100,
        batch=4,
        imgsz=640,
        device=0,
        project=WORK_DIR,
        name="train_exp",

        # Data augmentation
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        degrees=20.0,
        translate=0.2,
        scale=0.7,
        shear=12.0,
        perspective=0.001,
        flipud=0.5,
        fliplr=0.5,
        mosaic=1.0,
        mixup=0.15,
        copy_paste=0.1,

        warmup_epochs=3.0,
        warmup_momentum=0.8,
        warmup_bias_lr=0.1,
        box=7.5,
        cls=0.5,
        dfl=1.5,
        label_smoothing=0.0,
        overlap_mask=True,
        mask_ratio=4,
        dropout=0.2,
        optimizer='AdamW',
        lr0=0.001,
        lrf=0.01,
        weight_decay=0.001,
        val=True,

        plots=True,            # Generate plots
        verbose=True           # Detailed output
    )
    backup_model_and_config(100, f"{WORK_DIR}/train_exp/weights/best.pt", f"{WORK_DIR}/{CONFIG_FILE}") #backup
    print("Training is complete and saved to:", f"{WORK_DIR}/train_exp/weights/best.pt")


def detect_model():
    model = YOLO(f"{WORK_DIR}/train_exp/weights/best.pt")

    results = model.predict(
        source=f"{DATA_DIR}/test/images",
        imgsz=640,
        conf=0.5,
        device=0,
        save=True,
        save_txt=True,
        project=WORK_DIR,
        name="detect_exp"
    )
    print("test is completed and results are saved to:", f"{WORK_DIR}/detect_exp")


def visualize_results():
    img_dir = f"{WORK_DIR}/detect_exp/"
    label_dir = f"{WORK_DIR}/detect_exp/labels/"
    output_dir = f"{WORK_DIR}/output/"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for img_file in os.listdir(img_dir):
        if img_file.endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(img_dir, img_file)
            label_path = os.path.join(label_dir, os.path.splitext(img_file)[0] + ".txt")
            img = cv2.imread(img_path)
            if img is None:
                print(f"Warning: Could not read image {img_path}")
                continue

            h, w = img.shape[:2]

            if os.path.exists(label_path):
                with open(label_path, "r") as f:
                    for line in f:
                        parts = line.split()
                        if len(parts) >= 5:  # Standard YOLO format has 5 values
                            class_id = float(parts[0])
                            x_center = float(parts[1])
                            y_center = float(parts[2])
                            width = float(parts[3])
                            height = float(parts[4])

                            # If confidence is available (6th value), use it
                            conf = float(parts[5]) if len(parts) >= 6 else 1.0

                            class_name = "CT" if int(class_id) == 0 else "T"
                            x1 = int((x_center - width / 2) * w)
                            y1 = int((y_center - height / 2) * h)
                            x2 = int((x_center + width / 2) * w)
                            y2 = int((y_center + height / 2) * h)
                            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

                            label = f"{class_name} {conf:.2f}" if len(parts) >= 6 else class_name
                            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                        else:
                            print(f"Warning: Line '{line.strip()}' in {label_path} has fewer than 5 values")

            cv2.imwrite(os.path.join(output_dir, img_file), img)
            print(f"Saved visualization results: {os.path.join(output_dir, img_file)}")

def main():
    setup_logging()
    if not os.path.exists(f"{DATA_DIR}/train/images") or not os.path.exists(f"{DATA_DIR}/valid/images"):
        print(f"Error: Please make sure the dataset is located at '{DATA_DIR}/train/images' 和 '{DATA_DIR}/valid/images' 中！")
        return

    print("Step 1: Create a configuration file...")
    create_config()

    print("Step 2: Training the YOLOv11 model...")
    train_model()

    print("Step 3: Detection on the test set...")
    detect_model()

    print("Step 4: Visualization of test results...")
    visualize_results()

    print("Finish! The final result is saved to:", f"{WORK_DIR}/output/")

if __name__ == "__main__":
    main()

Step 1: 創建配置文件...
配置文件已創建: dataset/data.yaml
Step 2: 訓練 YOLOv11 模型...
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.113 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=dataset/data.yaml, epochs=100, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=dataset, name=train_exp5, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=N

train: Scanning /content/CSGO-1/train/labels.cache... 632 images, 247 backgrounds, 0 corrupt: 100%|██████████| 632/632 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 10, len(boxes) = 658. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 313.5±64.5 MB/s, size: 44.7 KB)


val: Scanning /content/CSGO-1/valid/labels.cache... 140 images, 59 backgrounds, 0 corrupt: 100%|██████████| 140/140 [00:00<?, ?it/s]


...

    100/100      1.82G      1.151      1.157      1.315          3        640: 100%|██████████| 158/158 [00:20<00:00,  7.66it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:01<00:00, 10.20it/s]

                   all        140        127      0.947      0.797      0.877      0.543



100 epochs completed in 0.696 hours.
Optimizer stripped from dataset/train_exp5/weights/last.pt, 5.5MB
Optimizer stripped from dataset/train_exp5/weights/best.pt, 5.5MB

Validating dataset/train_exp5/weights/best.pt...
Ultralytics 8.3.113 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  7.75it/s]


                   all        140        127      0.946      0.797      0.877      0.543
                    CT         63         98      0.942      0.835       0.92       0.53
                     T         19         29       0.95      0.759      0.835      0.556
Speed: 0.5ms preprocess, 6.3ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to dataset/train_exp5
Backup of model and config completed: backup/epoch_100_20250422_103506
訓練完成，模型權重保存至: dataset/train_exp/weights/best.pt
Step 3: 在測試集上檢測...

image 1/28 /content/CSGO-1/test/images/00000028_jpg.rf.df7a6a5316d8f82d68a210b53f4158d2.jpg: 640x640 (no detections), 8.7ms
image 2/28 /content/CSGO-1/test/images/00000033_jpg.rf.41549cdd85fc59bedceed8bc0e3973be.jpg: 640x640 (no detections), 10.3ms
image 3/28 /content/CSGO-1/test/images/00000041_jpg.rf.1bf4b22ac5665b3af44c8de8faca642a.jpg: 640x640 (no detections), 8.9ms
image 4/28 /content/CSGO-1/test/images/00000049_jpg.rf.b73bd51a87d34bcedb710278dd68253b.jpg: 640x640 (n

In [ ]:
import os

def remap_labels(label_dir, mapping):
    for label_file in os.listdir(label_dir):
        if label_file.endswith(".txt"):
            file_path = os.path.join(label_dir, label_file)
            with open(file_path, "r") as f:
                lines = f.readlines()
            with open(file_path, "w") as f:
                for line in lines:
                    parts = line.strip().split()
                    if not parts:
                        continue
                    old_id = int(parts[0])
                    new_id = mapping.get(old_id, -1)
                    if new_id != -1:
                        parts[0] = str(new_id)
                        f.write(" ".join(parts) + "\n")

mapping = {
    4: 0,
    6: 0,
    10: 1,
    11: 1,
    12: 1
}

label_dirs = ["/content/CSGO-1/train/labels", "/content/CSGO-1/valid/labels", "/content/CSGO-1/test/labels"]
for label_dir in label_dirs:
    if os.path.exists(label_dir):
        remap_labels(label_dir, mapping)
        print(f"Corrected label files in {label_dir}")
    else:
        print(f"Warning: {label_dir} does not exist, please confirm the path")

已修正標籤文件在 /content/CSGO-1/train/labels
已修正標籤文件在 /content/CSGO-1/valid/labels
已修正標籤文件在 /content/CSGO-1/test/labels


In [ ]:
import os

img_dir = "/content/CSGO-1/train/images"
label_dir = "/content/CSGO-1/train/labels"

missing_images = []
for label_file in os.listdir(label_dir):
    if label_file.endswith(".txt"):
        img_file = label_file.replace(".txt", ".jpg")
        img_path = os.path.join(img_dir, img_file)
        if not os.path.exists(img_path):
            missing_images.append(img_file)

if missing_images:
    print("The following images are missing:", missing_images)
else:
    print("All images matching the label")

所有圖片與標籤匹配


In [ ]:
def image_predict(model_path, image_path, output_path=None, conf_threshold=0.25):
    """
    Predict objects in a single image and visualize the results.

    Args:
        model_path: Path to the trained YOLO model
        image_path: Path to the input image
        output_path: Path to save the output image (if None, will use 'output_image.jpg')
        conf_threshold: Confidence threshold for detections

    Returns:
        Path to the saved output image
    """
    if output_path is None:
        output_path = "output_image.jpg"

    # Ensure output directory exists
    output_dir = os.path.dirname(output_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Load model
    model = YOLO(model_path)

    # Perform prediction
    results = model.predict(source=image_path, conf=conf_threshold, save=False)

    # Load the original image
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")

    h, w = img.shape[:2]

    # Process the results
    for result in results:
        boxes = result.boxes
        for i, box in enumerate(boxes):
            # Get box coordinates
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)

            # Get class and confidence
            class_id = int(box.cls[0].item())
            confidence = box.conf[0].item()

            # Draw bounding box
            class_name = "CT" if class_id == 0 else "T"
            color = (0, 255, 0) if class_id == 0 else (0, 0, 255)  # Green for CT, Red for T
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

            # Add label
            label = f"{class_name} {confidence:.2f}"
            (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(img, (x1, y1 - text_height - 10), (x1 + text_width, y1), color, -1)
            cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Save the annotated image
    cv2.imwrite(output_path, img)
    print(f"Prediction completed. Output saved to: {output_path}")

    return output_path

def video_predict(model_path, video_path, output_path=None, conf_threshold=0.25, fps=None):
    """
    Predict objects in a video and create a new video with visualized results.

    Args:
        model_path: Path to the trained YOLO model
        video_path: Path to the input video
        output_path: Path to save the output video (if None, will use 'output_video.mp4')
        conf_threshold: Confidence threshold for detections
        fps: Output video FPS (if None, will use the same as input video)

    Returns:
        Path to the saved output video
    """
    if output_path is None:
        output_path = "output_video.mp4"

    # Ensure output directory exists
    output_dir = os.path.dirname(output_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Load model
    model = YOLO(model_path)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video file: {video_path}")

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    if fps is None:
        fps = original_fps

    # Set up video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID'
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            if frame_count % 10 == 0:
                print(f"Processing frame {frame_count}/{total_frames} ({frame_count/total_frames*100:.1f}%)")

            # Perform prediction on the frame
            results = model.predict(source=frame, conf=conf_threshold, save=False)

            # Process detection results
            for result in results:
                boxes = result.boxes
                for i, box in enumerate(boxes):
                    # Get box coordinates
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)

                    # Get class and confidence
                    class_id = int(box.cls[0].item())
                    confidence = box.conf[0].item()

                    # Draw bounding box
                    class_name = "CT" if class_id == 0 else "T"
                    color = (0, 255, 0) if class_id == 0 else (0, 0, 255)  # Green for CT, Red for T
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                    # Add label
                    label = f"{class_name} {confidence:.2f}"
                    (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                    cv2.rectangle(frame, (x1, y1 - text_height - 10), (x1 + text_width, y1), color, -1)
                    cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # Write the frame to the output video
            out.write(frame)

        print(f"Video processing completed. Output saved to: {output_path}")

    except Exception as e:
        print(f"Error processing video: {str(e)}")

    finally:
        # Release resources
        cap.release()
        out.release()

    return output_path

image_predict(model_path='/content/dataset/train_exp5/weights/best.pt', image_path='/content/drive/MyDrive/test.png')
video_predict(model_path='/content/dataset/train_exp5/weights/best.pt', video_path='/content/drive/MyDrive/Valorant 2025.03.13 - 12.29.10.02.DVR.mp4')


image 1/1 /content/drive/MyDrive/test.png: 256x640 1 T, 54.6ms
Speed: 1.9ms preprocess, 54.6ms inference, 1.4ms postprocess per image at shape (1, 3, 256, 640)
Prediction completed. Output saved to: output_image.jpg

0: 384x640 (no detections), 9.7ms
Speed: 2.3ms preprocess, 9.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 14.6ms
Speed: 4.0ms preprocess, 14.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 18.8ms
Speed: 3.6ms preprocess, 18.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 14.3ms
Speed: 5.3ms preprocess, 14.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.6ms
Speed: 3.3ms preprocess, 13.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.7ms
Speed: 3.8ms preprocess, 13.7ms inference, 0.7ms postprocess per image at shape (1, 

'output_video.mp4'

yolov5 and v8

In [ ]:
import os
import cv2
import torch
from ultralytics import YOLO
import shutil
import yaml
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

!pip install ultralytics roboflow -qqq

from roboflow import Roboflow
rf = Roboflow(api_key="TSNpeVNs514e85awM4WA")
project = rf.workspace("pg-g-sgpnk").project("csgo-wlnz5")
version = project.version(1)
dataset = version.download("yolov8")

WORK_DIR = "dataset"
DATA_DIR = "/content/CSGO-1"
CONFIG_FILE = "data.yaml"

if not os.path.exists(WORK_DIR):
    os.makedirs(WORK_DIR)

def create_config():
    config_content = f"""
train: {DATA_DIR}/train/images
val: {DATA_DIR}/valid/images
test: {DATA_DIR}/test/images
nc: 2
names: ['CT', 'T']
    """
    with open(f"{WORK_DIR}/{CONFIG_FILE}", "w") as f:
        f.write(config_content.strip())
    print(f"Configuration file created:{WORK_DIR}/{CONFIG_FILE}")

def check_weights_file(weights):
    """Check if the weights file exists or is valid."""
    try:
        model = YOLO(weights)
        return True
    except FileNotFoundError:
        print(f"Error: Weights file '{weights}' not found.")
        return False
    except Exception as e:
        print(f"Error loading weights '{weights}': {str(e)}")
        return False

def train_model(model_name, weights):
    if not check_weights_file(weights):
        print(f"Skipping training for {model_name} due to invalid weights.")
        return None
    model = YOLO(weights)
    results = model.train(
        data=f"{WORK_DIR}/{CONFIG_FILE}",
        epochs=30,
        batch=16,
        imgsz=640,
        device=0 if torch.cuda.is_available() else 'cpu',
        project=WORK_DIR,
        name=f"{model_name}_exp",
        exist_ok=True,
        patience=5,
        augment=True,
        lr0=0.001,
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        degrees=20.0,
        translate=0.1,
        scale=0.5,
        shear=10,
        flipud=0.5,
        fliplr=0.5,
        mosaic=1.0,
        mixup=0.1,
        erasing=0.4
    )
    print(f"{model_name} Training is complete and weights are saved to: {WORK_DIR}/{model_name}_exp/weights/best.pt")
    return results

def evaluate_model(model, source):
    results = model.predict(
        source=source,
        imgsz=640,
        conf=0.5,
        device=0 if torch.cuda.is_available() else 'cpu',
        save_txt=True,
        project=WORK_DIR,
        name="eval_exp",
        exist_ok=True
    )
    map50 = results[0].metrics.mAP50 if hasattr(results[0], 'metrics') and hasattr(results[0].metrics, 'mAP50') else 0.0
    return map50

def visualize_results(model, image_path, output_dir="output"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image {image_path}")
        return
    results = model.predict(image_path, conf=0.5)
    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0]
        cls = int(box.cls[0])
        label = f"{'CT' if cls == 0 else 'T'} {conf:.2f}"
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    output_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(output_path, img)
    print(f"visualize_results saved to: {output_path}")

def main():
    create_config()
    models = [
        {"name": "yolov5", "weights": "yolov5s.pt"},
        {"name": "yolov8", "weights": "yolov8n.pt"}
    ]
    map_results = {}
    for model_info in models:
        print(f"\n Train {model_info['name']}...")
        train_model(model_info['name'], model_info['weights'])

        model_path = f"{WORK_DIR}/{model_info['name']}_exp/weights/best.pt"
        if not os.path.exists(model_path):
            print(f"Error: Trained weights not found for {model_info['name']}")
            continue
        model = YOLO(model_path)

        test_source = f"{DATA_DIR}/test/images"
        map50 = evaluate_model(model, test_source)
        map_results[model_info['name']] = map50
        test_images = [os.path.join(test_source, img) for img in os.listdir(test_source) if img.endswith((".jpg", ".png"))]
        if test_images:
            print(f"\n Visualize {model_info['name']} Example image:")
            visualize_results(model, test_images[0], f"output/{model_info['name']}")

if __name__ == "__main__":
    main()